In [2]:
import json
import requests
import time

### should be outside the repo
cred_location = r'/Users/jj/code/strava_creds/'

In [7]:
def get_client_tokens():
    print('please go here https://www.strava.com/settings/api, and copy the token information as prompted')
    client_id = input('please enter your client id')
    client_secret = input('please enter your client secret')
    redirect_uri = input('please enter a redirect uri (use http://localhost/ if unsure)')
    client_tokens = {
        'client_id':str(client_id),
        'client_secret':str(client_secret),
        'redirect_uri':str(redirect_uri)
    }

    with open(cred_location+'client_tokens.json', 'w') as outfile:
        json.dump(client_tokens, outfile, indent=4)

    print("client tokens saved")
    return None

### This will just check if the client tokens exist, not if they are valid.
### client token, client secret, and redirect_uri should all be held in a json file called client_tokens.json
def check_client_tokens():
    try:
        client_tokens = json.load(open(cred_location+'client_tokens.json'))
        if {'client_id', 'client_secret', 'redirect_uri'} == set(client_tokens.keys()):
            print('Client Tokens Found')
            return True
        else:
            return False
    except:
        return False

if check_client_tokens() == False:
    get_client_tokens()

Client Tokens Found


In [8]:
def authorize():

      client_tokens = json.load(open(cred_location+'client_tokens.json'))

      # Authorization URL
      request_url = f'http://www.strava.com/oauth/authorize?client_id={client_tokens["client_id"]}' \
                        f'&response_type=code&redirect_uri={client_tokens["redirect_uri"]}' \
                        f'&approval_prompt=force' \
                        f'&scope=profile:read_all,activity:read_all'

      # User prompt showing the Authorization URL
      # and asks for the code
      print('Click here:', request_url)
      print('Please authorize the app and copy&paste below the generated code!')
      print('P.S: you can find the code in the URL')
      code = input('Insert the code from the url: ')

      # Get the access token
      token = requests.post(url='https://www.strava.com/api/v3/oauth/token',
                        data={'client_id': client_tokens["client_id"],
                              'client_secret': client_tokens["client_secret"],
                              'code': code,
                              'grant_type': 'authorization_code'})

      access_tokens = token.json()
      

      with open(cred_location+'access_tokens.json', 'w') as outfile:
            json.dump(access_tokens, outfile, indent=4)
      print('Authorisation Complete')


def check_authorization():
      try:
            tokens = json.load(open(cred_location+'access_tokens.json'))
            if time.time() < int(tokens['expires_at']) and 'access_token' in tokens.keys():
                  print((int(tokens['expires_at']) - int(time.time())) / 60, 'minutes until token expires')
                  print('Unexpired Token Found')
                  return True
            else:
                  return False
      except:
            return False

if check_authorization() == False:
      authorize()

359.85 minutes until token expires
Unexpired Token Found
